In [7]:
import imutils
import numpy as np
from PIL import Image, ImageOps
from collections import deque
import os
import random
import csv

In [8]:
def bfs(coordinates, visited, start, num):
    directions = [(1, 0), (-1, 0), (0, 1), (0, -1)]
    queue = deque([start])
    area = 0

    while queue:
        current = queue.popleft()
        x, y = current

        if visited[x][y] == 0:
            visited[x][y] = num
            area += 1

            for dx, dy in directions:
                nx, ny = x + dx, y + dy
                if (nx, ny) in coordinates and visited[nx][ny] == 0:
                    queue.append((nx, ny))

    return area


def count_black_areas(coordinates):
    if not coordinates:
        return []

    min_x, min_y = min(x for x, y in coordinates), min(
        y for x, y in coordinates)
    max_x, max_y = max(x for x, y in coordinates), max(
        y for x, y in coordinates)

    rows, cols = max_x - min_x + 1, max_y - min_y + 1
    visited = [[0] * cols for _ in range(rows)]
    mapped_coordinates = [(x - min_x, y - min_y) for x, y in coordinates]

    num = 1
    areas = []
    area_mapping = {}

    for i in range(rows):
        for j in range(cols):
            if (i, j) in mapped_coordinates and visited[i][j] == 0:
                area = bfs(mapped_coordinates, visited, (i, j), num)
                areas.append(area)
                current_area_coordinates = [
                    (x + min_x, y + min_y) for x, y in mapped_coordinates if visited[x][y] == num]
                area_mapping[num] = current_area_coordinates
                num += 1

    return areas, area_mapping


def extract_all_non_white_points(image_path, num_points=68, test=False, radius=1):
    with Image.open(image_path) as img:
        img = ImageOps.flip(img)
        if img.mode != 'RGB':
            img = img.convert('RGB')
        img = np.array(img)

        img = imutils.resize(img, width=500)
        pixels = np.array(img)

    white_threshold = 245
    non_white_pixels = np.where(np.any(pixels < white_threshold, axis=-1))
    non_white_point_coordinates = list(zip(non_white_pixels[1], non_white_pixels[0]))

    """
    points = []
    for x, y in non_white_point_coordinates:
        neighborhood = pixels[y - radius:y + radius + 1, x - radius:x + radius + 1]
        if np.any(neighborhood >= white_threshold):
            points.append((x, y))

    if test:
        plt.scatter(*zip(*points), label='non_white_points')
        plt.title('ori_kaomoji')
        plt.show()

    if len(points) < num_points:
        raise ValueError("Not enough non-white points to sample the desired number of points.")
    sampled_points = random.sample(points, num_points)
    """
    
    #bfs找出每個色塊的面積然後按比例random sample
    areas, area_mapping = count_black_areas(non_white_point_coordinates)
    sampled_points = []
    total_area = sum(areas)
    ratio = [int(num_points * area / total_area) for area in areas] 
    diff = num_points - sum(ratio)
    max_ratio_index = ratio.index(max(ratio))
    ratio[max_ratio_index] += diff

    for area, selection in zip(area_mapping, ratio):
        if area in area_mapping:
            sampled_points.extend(random.sample(area_mapping[area], selection))
    return sampled_points

In [9]:
folder_path = 'images'
csv_file_path = 'points.csv'
image_files = [f for f in os.listdir(folder_path) if f.endswith(('.jpg', '.png'))]
with open(csv_file_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    # 寫入標頭
    header = ['File Name'] + [f'Point {i}' for i in range(1, 69)]
    writer.writerow(header)
    
    # 迭代每一個檔案，提取點，並寫入CSV
    random.seed(42)
    for image_file in image_files:
        image_path = os.path.join(folder_path, image_file)
        points = extract_all_non_white_points(image_path)
        #print(points)
        # 將每個點的(x,y)座標轉換成"(x,y)"格式的字符串
        points_str = [f"({x},{y})" for x, y in points]
        # 寫入檔名和所有點的座標
        writer.writerow([image_file] + points_str)